[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/russellpwirtz/llm-playground/blob/main/wizard-coder-colab.ipynb)

In [ ]:
# Setup

%cd /content
!apt-get -y install -qq aria2

!git clone -b v1.3 https://github.com/camenduru/text-generation-webui
%cd /content/text-generation-webui
!pip install -r requirements.txt
!pip install -U gradio==3.28.3

!mkdir /content/text-generation-webui/repositories
%cd /content/text-generation-webui/repositories
!git clone -b cuda https://github.com/camenduru/GPTQ-for-LLaMa.git
%cd GPTQ-for-LLaMa
!python setup_cuda.py install

%cd /content/text-generation-webui


In [ ]:
# Model download

# MODEL_NAME='WizardCoder-15B-1.0-GPTQ'
MODEL_NAME='WizardCoder-Guanaco-15B-V1.1-GPTQ'

REPO_NAME = f'https://huggingface.co/TheBloke/{MODEL_NAME}'
SAFETENSORS_FILE = 'gptq_model-4bit-128g.safetensors'

print('Downloading model files...')
files = ['config.json', 'generation_config.json', 'special_tokens_map.json', 'added_tokens.json', 'tokenizer_config.json', 'quantize_config.json', 'generation_config.json', 'tokenizer.json','vocab.json', ]

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M '{REPO_NAME}/resolve/main/{SAFETENSORS_FILE}' -d 'models/{MODEL_NAME}' -o '{SAFETENSORS_FILE}'

for file in files:
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M '{REPO_NAME}/raw/main/{file}' -d 'models/{MODEL_NAME}' -o '{file}'
  

In [ ]:
# Start app

!python server.py --share --chat --wbits 4 --groupsize 128 --model_type llama --model {MODEL_NAME}